##  Time distance to services

Finds the time and distance of each mesh block to the nearest service, by a specified mode.

Inputs: A mesh block input polygon dataset; A set of services (each as a point dataset); A set of travel modes (in network-mode pairs)

Output: A mesh block polygon datset with additionally attributes containing the time and distance to each service, by each mode

In [1]:
import arcpy
from arcgis import GeoAccessor
import pandas as pd

### Specify the services we want to look consider


In [2]:
services_list = ["Services\\CQ_TownCentres", "Services\\CQ_GroupCentres", "Services\\CQ_LocalCentres", "Services\\CQ_Schools"]

In [3]:
#specify the input mesh block (this input data is not edited)

MB_input = "MB_test" #(for testing) # "MB16_Time_Distance" # 

### Run the tool

In [4]:
#all_services_TimeDistance(services_list, MB_Layer = MB_input)#, MB_Output_Name = "MB16_TD_test")#TD stands for Time-Distance

##### TO DO: Add multi-network functionality

In [5]:
# specify path of paths_ND

In [6]:
# arcpy.conversion.TableToTable(r"OD Cost Matrix\Lines", r"C:\Users\gjames\OneDrive - Esri Australia\Honours\Data\OD Data Production 200622", "Lines_Walk__CQ_TownCentres.csv", '', r'Name "Name" true true true 1024 Text 0 0,First,#,OD Cost Matrix 4\Lines,Name,0,1024;OriginID "OriginID" true true true 4 Long 0 0,First,#,OD Cost Matrix 4\Lines,OriginID,-1,-1;DestinationID "DestinationID" true true true 4 Long 0 0,First,#,OD Cost Matrix 4\Lines,DestinationID,-1,-1;DestinationRank "DestinationRank" true true true 4 Long 0 0,First,#,OD Cost Matrix 4\Lines,DestinationRank,-1,-1;Total_Cycle_Time "Total_Cycle_Time" true true true 8 Double 0 0,First,#,OD Cost Matrix 4\Lines,Total_Cycle_Time,-1,-1;Total_ECycle_Time "Total_ECycle_Time" true true true 8 Double 0 0,First,#,OD Cost Matrix 4\Lines,Total_ECycle_Time,-1,-1;Total_Walk_Time "Total_Walk_Time" true true true 8 Double 0 0,First,#,OD Cost Matrix 4\Lines,Total_Walk_Time,-1,-1;Total_Length "Total_Length" true true true 8 Double 0 0,First,#,OD Cost Matrix 4\Lines,Total_Length,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OD Cost Matrix 4\Lines,Shape_Length,-1,-1', '')

In [7]:
def all_services_TimeDistance(Services_List, Network_Dataset = "TransitNetwork_ND", Mode = "Public transit time", MB_Layer = "MB16_Time_Distance", MB_Output_Name = "MB_Layer_temp"):
    '''Adds the time and distance to each of the services given to a field in the mesh_block_layer
    Inputs:
        Network Dataset name (str); Mode (str); Services_list (list); 
        MB_Layer - A meshblock layer to run closest facility on
        MB_Output_Name - The name of the updated MB_layer to put the results in
    Outputs: 
        (Implicit) MB_Layer_temp - A new MB layer containing the times and distances as new attributes 
    '''
    
    #create a copy of the MB layer to store the joins from each closest facility
    #this allows for multiple runs
    MB_Layer = arcpy.conversion.FeatureClassToFeatureClass(in_features=MB_Layer, out_name=MB_Output_Name, where_clause="", config_keyword="")[0]
    
    for service in Services_List:
        #run closest facility
        #gives a different name for each service and network mode because of join issues
        closest_facility = Closest_Facility_To_Centres(Incidents_Polygons = MB_Layer, Network_Dataset = Network_Dataset, Mode = Mode, Input_Facilities = service, Output_Layer_Name= Mode[:4] + service)
        
        ##add the time and distance to the MB_Layer
  
        #these will be in the form "Total_Impedance" and "Total_Length" fields (except public transit)
        #manually specify impedance
        impedance = {"Public transit time" : "PublicTransitTime", "Walking": "Walk_Time", "Cycling": "Cycle_Time", "Ecycling": "Ecycle_Time"}
        
        transfer_fields = ["Total_"+ impedance[Mode], "Total_Length"]
        
        #join the required field from closest facility
        
        arcpy.management.JoinField(in_data=MB_Layer, in_field="OBJECTID", join_table=closest_facility.name + "\\Routes", join_field="IncidentID", fields=transfer_fields)[0]
        
        #estabish field names
        time_field_name = service.split("\\")[-1][3:] + "_Time_" + Mode[:4] #exclude "xx\\CQ_" from service name, and give only first 7 char of the mode 
        distance_field_name = service.split("\\")[-1][3:] + "_Distance_" + Mode[:4]
        
        #rename the field (also allows data to be saved)
        arcpy.management.AlterField(in_table=MB_Layer, field = transfer_fields[0], new_field_name = time_field_name, new_field_alias = time_field_name)
        arcpy.management.AlterField(in_table=MB_Layer, field = transfer_fields[1], new_field_name = distance_field_name, new_field_alias = distance_field_name)
     

In [8]:
#closest facility from Population accessability file
def Closest_Facility_To_Centres(Incidents_Polygons="MB16_CQ_empl", Network_Dataset="TransitNetwork_ND", Mode = "Public transit time", Cutoff=None, Input_Facilities="LR1_Stops", Output_Layer_Name = "Closest Facility"):  # Closest_Facility_To_Centres
    '''Runs the closest facility tool from a single function. 
    Inputs: A polygon layer for the incidents, a network dataset, a cutoff time in minutes, point input facilities, and a string name for 
    the output layer.
    Outputs: A closest facility network analysis layer
    Assumptions: A fixed start time at Thursday 11am.
    '''
    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Process: Make Closest Facility Analysis Layer (Make Closest Facility Analysis Layer) (na)
    Closest_Facility = arcpy.na.MakeClosestFacilityAnalysisLayer(network_data_source=Network_Dataset, layer_name=Output_Layer_Name, travel_mode=Mode, travel_direction="TO_FACILITIES", cutoff=Cutoff, number_of_facilities_to_find=1, time_of_day="4/01/1900 11:00:00 AM", time_zone="LOCAL_TIME_AT_LOCATIONS", time_of_day_usage="END_TIME", line_shape="ALONG_NETWORK", accumulate_attributes=[], generate_directions_on_solve="NO_DIRECTIONS", ignore_invalid_locations="SKIP")[0]

    # Process: Add Locations (Facilities) (Add Locations) (na)
    Closest_Facility_with_Facilities_ = arcpy.na.AddLocations(in_network_analysis_layer=Closest_Facility, sub_layer="Facilities", in_table=Input_Facilities, field_mappings="", search_tolerance="5000 Meters", sort_field="", search_criteria=[["StopConnectors", "SHAPE"], ["Streets", "SHAPE"], ["Stops", "SHAPE"], ["StopsOnStreets", "SHAPE"], ["TransitNetwork_ND_Junctions", "NONE"]], match_type="MATCH_TO_CLOSEST", append="APPEND", snap_to_position_along_network="NO_SNAP", snap_offset="5 Meters", exclude_restricted_elements="EXCLUDE", search_query=[])[0]

    # Process: Feature To Point (Feature To Point) (management)
    Incident_Points = "Incidents_points"
    arcpy.management.FeatureToPoint(in_features=Incidents_Polygons, out_feature_class=Incident_Points, point_location="CENTROID")  

    # Process: Add Locations (Incidents) (Add Locations) (na)
    Closest_Facility_with_Facilities_and_Incidents_ = arcpy.na.AddLocations(in_network_analysis_layer=Closest_Facility_with_Facilities_, sub_layer="Incidents", in_table=Incident_Points, field_mappings="", search_tolerance="5000 Meters", sort_field="", search_criteria=[["StopConnectors", "SHAPE"], ["Streets", "SHAPE"], ["Stops", "SHAPE"], ["StopsOnStreets", "SHAPE"], ["TransitNetwork_ND_Junctions", "NONE"]], match_type="MATCH_TO_CLOSEST", append="APPEND", snap_to_position_along_network="NO_SNAP", snap_offset="5 Meters", exclude_restricted_elements="EXCLUDE", search_query=[])[0]

    # Process: Solve (Solve) (na)
    Closest_Facility_solved, Solve_Succeeded = arcpy.na.Solve(in_network_analysis_layer=Closest_Facility_with_Facilities_and_Incidents_, ignore_invalids="SKIP", terminate_on_solve_error="TERMINATE", simplification_tolerance="", overrides="")
    
    return Closest_Facility_solved

## Exporting Data

Here, important data can be exported for further processing, or other uses.

#### Specifying the parameters of what data to export and where

In [44]:
#general location of data (keep this constant) ("C:/Users/gjames/OneDrive - Esri Australia/Honours/Data/")
path = "C:/Users/gjames/OneDrive - Esri Australia/Honours/Data/"

In [45]:
# folder to store the data in (e.g. "Testing")
folder = "Testing"

In [50]:
# the feature layer name, from this project (e.g. "MB16_CQ_empl", "Services/CQ_TownCentres")
feature_layer_name = "Services/CQ_TownCentres"

#### Export the given spatial data to a csv

In [64]:
#### Export the given spatial data
sdf = GeoAccessor.from_featureclass(feature_layer_name)

#ensuring that to_csv doesn't try to create a new folder
if "/" in feature_layer_name:
    output_feature_layer_name = feature_layer_name.replace('/', "-")
    
# output to csv
sdf.to_csv(path + folder + "/" + output_feature_layer_name + ".csv")

In [52]:
feature_layer_name

'Services/CQ_TownCentres'

In [56]:
if "/" in feature_layer_name:
    print(feature_layer_name.replace('/', "-"))

Services-CQ_TownCentres


#### In function form

In [62]:
def export_to_csv(feature_layer_name, folder = "Testing", path = "C:/Users/gjames/OneDrive - Esri Australia/Honours/Data/"):
    '''
    Exports the attribute layer of the given feature layer name to a preexisting given folder at the given path
    '''
    #convert given data into a dataframe
    sdf = GeoAccessor.from_featureclass(feature_layer_name)

    #ensuring that to_csv doesn't try (and fail) to create a new folder
    if "/" in feature_layer_name:
        output_feature_layer_name = feature_layer_name.replace('/', "-")
    if "\\" in feature_layer_name:
        output_feature_layer_name = feature_layer_name.replace('\\', "-")
    
    # output to csv
    sdf.to_csv(path + folder + "/" + output_feature_layer_name + ".csv")

## Origin Destination Matrix Code

Runs the origin destination matrix tool for the specified input point data.

In [9]:
def OD_data_production(layers, Mode):
    '''
    Finds the travel times between each of the given origins (MB data) and destinations (various service data) using the 
    Origin Destination Cost Matrix tool.
    
    Inputs: layers - a list of the names of point data to run the tool on (list of strings), note - these layers must be contained
            MainDatasets.gdb example: layers = ["CQ_TownCentres", "CQ_GroupCentres"]. Could implement an optional path specification
                
            Mode - The mode to be used in the analysis ("Walking", "Cycling" or "Public transit time")
                The network dataset to be used is set based of the mode as one of:
                    "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND"
                    "C:/Users/gjames/Documents/ArcGIS/Projects/GTFS_Integrated/GTFS_Integrated.gdb/GTFS_CQ/TransitNetwork_ND"
            
    Outputs: A .csv file for each of the input layers, in the Honours/Data/OD Data Production folder
    '''
    
    arcpy.env.overwriteOutput = True #always important to include - allows tool to be run more than once without creating extra layers
    
    #initialise the network analysis layer

    if Mode == "Walking" or Mode == "Cycling":
        Network_Dataset = "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND"
        
        arcpy.na.MakeODCostMatrixAnalysisLayer(Network_Dataset, "OD Cost Matrix", Mode, line_shape = "NO_LINES") 
        
        
    elif Mode == "Public transit time":
        Network_Dataset = "C:/Users/gjames/Documents/ArcGIS/Projects/GTFS_Integrated/GTFS_Integrated.gdb/GTFS_CQ/TransitNetwork_ND"
        
        # this is done separately as adding a time of day to Paths_ND signficantly adds computation time
        arcpy.na.MakeODCostMatrixAnalysisLayer(Network_Dataset, "OD Cost Matrix", Mode,
                                           time_of_day = '4/1/1900 11:00 AM', line_shape = "NO_LINES")
        
        
    else:
        return 'Invalid Mode, choose one of "Walking", "Cycling" or "Public transit time"'
        #time_of_day = '1/4/1900 11:00 AM'
    

    
    #add the population data (constant for all runs)
    arcpy.na.AddLocations("OD Cost Matrix", "Origins", 
                          r"C:\Users\gjames\Documents\ArcGIS\Projects\MainDatasets\MainDatasets.gdb\MB16_CQ_pop_points")

    
    #run the process for each of the layers
    for layer in layers:
        
        #add locations
        destinations_path = "C:/Users/gjames/Documents/ArcGIS/Projects/MainDatasets/MainDatasets.gdb/" + layer
        
        arcpy.na.AddLocations("OD Cost Matrix", "Destinations",
                              destinations_path, append = "CLEAR")
        
        #solve
        arcpy.na.Solve("OD Cost Matrix", "SKIP", "TERMINATE", None, '')
        
        #export to csv
        output_name = "Lines_" + Mode + "__" + layer + ".csv"
        output_location = r"C:\Users\gjames\OneDrive - Esri Australia\Honours\Data\OD Data Production"
        
        arcpy.conversion.TableToTable(r"OD Cost Matrix\Lines", output_location, output_name)
    
    
    

In [10]:
layers = ["CQ_LocalCentres", "Nonzero_MB16_CQ_empl_points"]

In [11]:
#OD_data_production(layers, "Public transit time")

In [12]:
#2021 mesh block version of the code - could set funtion to have an option later, but copy pasting is faster for now
#only changes population source data, and output name
def OD_data_production21(layers, Mode, services_path = "C:/Users/gjames/Documents/ArcGIS/Projects/MainDatasets/MainDatasets.gdb/"):
    '''
    Finds the travel times between each of the given origins (MB data) and destinations (various service data) using the 
    Origin Destination Cost Matrix tool.
    
    Inputs: layers - a list of the names of point data to run the tool on (list of strings), note - these layers should be contained
            within MainDatasets.gdb, or otherwise the path to the services should be manually specified example: layers = 
            ["CQ_TownCentres", "CQ_GroupCentres"]
                
            Mode - The mode to be used in the analysis ("Walking", "Cycling" or "Public transit time")
                The network dataset to be used is set based of the mode as one of:
                    "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND"
                    "C:/Users/gjames/Documents/ArcGIS/Projects/GTFS_Integrated/GTFS_Integrated.gdb/GTFS_CQ/TransitNetwork_ND"
            
    Outputs: A .csv file for each of the input layers, in the Honours/Data/OD Data Production folder
    '''
    
    arcpy.env.overwriteOutput = True #always important to include - allows tool to be run more than once without creating extra layers
    
    #initialise the network analysis layer

    if Mode == "Walking" or Mode == "Cycling":
        Network_Dataset = "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND"
        
        arcpy.na.MakeODCostMatrixAnalysisLayer(Network_Dataset, "OD Cost Matrix", Mode, line_shape = "NO_LINES") 
        
        
    elif Mode == "Public transit time":
        Network_Dataset = "C:/Users/gjames/Documents/ArcGIS/Projects/GTFS_Integrated/GTFS_Integrated.gdb/GTFS_CQ/TransitNetwork_ND"
        
        # this is done separately as adding a time of day to Paths_ND signficantly adds computation time
        arcpy.na.MakeODCostMatrixAnalysisLayer(Network_Dataset, "OD Cost Matrix", Mode,
                                           time_of_day = '4/1/1900 11:00 AM', line_shape = "NO_LINES")
        
        
    else:
        return 'Invalid Mode, choose one of "Walking", "Cycling" or "Public transit time"'
        #time_of_day = '1/4/1900 11:00 AM'
    

    
    #add the population data (constant for all runs)
    arcpy.na.AddLocations("OD Cost Matrix", "Origins", 
                          r"C:\Users\gjames\Documents\ArcGIS\Projects\MainDatasets\MainDatasets.gdb\MB21_CQ_pop_points")

    
    #run the process for each of the layers
    for layer in layers:
        
        #add locations
        destinations_path = services_path + layer
        
        arcpy.na.AddLocations("OD Cost Matrix", "Destinations",
                              destinations_path, append = "CLEAR")
        
        #solve
        arcpy.na.Solve("OD Cost Matrix", "SKIP", "TERMINATE", None, '')
        
        #export to csv
        output_name = "Lines21_" + Mode + "__" + layer + ".csv"
        output_location = r"C:\Users\gjames\OneDrive - Esri Australia\Honours\Data\OD Data Production"
        
        arcpy.conversion.TableToTable(r"OD Cost Matrix\Lines", output_location, output_name)
    

In [8]:
layers = ["CQ_Schools"]
#layers = ["CQ_TownCentres", "CQ_LocalCentres","CQ_Schools", "CQ_GroupCentres", "Nonzero_MB16_CQ_empl_points"]

In [4]:
#OD_data_production21(layers, "Walking")
OD_data_production21(layers, "Walking")

In [19]:
#OD_data_production21(layers, "Walking")

In [13]:
LA_layers = ["LAma100_Schools_mdINIS"] #dev_layers[:]
services_path = "C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\Development_Scenarios\\Development_Scenarios.gdb\\"

In [14]:
#OD_data_production21(layers, "Walking")
OD_data_production21(LA_layers, "Walking", services_path = services_path)

In [31]:
LA_layers

'LA results\\LAma100_GroupCentres_mdINIS'

In [37]:
dev_layers[1]

'LAma100_GroupCentres_mdINIS'

In [15]:
#get all the layers from the development scenarios project
dev_layers = []

proj = arcpy.mp.ArcGISProject(r'C:\Users\gjames\Documents\ArcGIS\Projects\Development_Scenarios\Development_Scenarios.aprx')
for layer in proj.listMaps()[0].listLayers():
    
    if str(layer).split('\\')[0] == "LA results":
        try:
            dev_layers.append(str(layer).split("\\")[1])
            print(str(layer).split("\\")[1])
            
        except:
            continue

LAma100_Schools_mdINIS
LAma100_GroupCentres_mdINIS
LAma100_LocalCentres_mdINIS
LAma100_Schools_Actual
LAma100_GroupCentres_Actual
LAma100_LocalCentres_Actual
LAma100_Schools_TODInfill
LAma100_GroupCentres_TODInfill
LAma100_LocalCentres_TODInfill
LAma100_Schools_Greenfield
LAma100_GroupCentres_Greenfield
LAma100_LocalCentres_Greenfield
LAma100_Schools_EqualDev
LAma100_GroupCentres_EqualDev
LAma100_LocalCentres_EqualDev


In [5]:
"C:/Users/gjames/Documents/ArcGIS/Projects/MainDatasets/MainDatasets.gdb/" + "CQ_LocalCentres"

'C:/Users/gjames/Documents/ArcGIS/Projects/MainDatasets/MainDatasets.gdb/CQ_LocalCentres'

In [ ]:
arcpy.na.MakeClosestFacilityAnalysisLayer(r"C:\Users\gjames\Documents\ArcGIS\Projects\GTFS_Integrated\GTFS_Integrated.gdb\GTFS_CQ\TransitNetwork_ND", "Closest Facility 2", "Public transit time", "TO_FACILITIES", None, 1, None, "LOCAL_TIME_AT_LOCATIONS", "START_TIME", "ALONG_NETWORK", None, "NO_DIRECTIONS", "SKIP")

In [ ]:
##production of data

#OD_data_production(layers, "Walking", "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND")





In [ ]:
arcpy.na.AddLocations("Closest Facility 2", "Incidents",
                      r"C:\Users\gjames\Documents\ArcGIS\Projects\MainDatasets\MainDatasets.gdb\MB16_CQ_pop_points",
                      "Name # #;TargetFacilityCount # #;CurbApproach # 0;Attr_PublicTransitTime # 0;Attr_WalkTime # 0;Attr_Length # 0;Cutoff_PublicTransitTime # #;Cutoff_WalkTime # #;Cutoff_Length # #",
                      "5000 Meters", None, "StopConnectors SHAPE;Streets SHAPE;Stops SHAPE;StopsOnStreets SHAPE;TransitNetwork_ND_Junctions NONE",
                      "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "5 Meters", "EXCLUDE", None)

## Testing

In [65]:
export_to_csv("Services\\CQ_LocalCentres")



sdf = GeoAccessor.from_featureclass("MB16_CQ_empl")

In [48]:
sdf_towncentres

,OBJECTID,MB_CODE16,MB_CAT16,Population_density,Person,State,ORIG_FID,SHAPE
0,1,80036983000,Commercial,7.015903,3.0,8.0,4269.0,"{""x"": 149.08546049210804, ""y"": -35.34473202895..."
1,2,None,None,NaN,NaN,NaN,NaN,"{""x"": 149.1296045511092, ""y"": -35.276486929628..."
2,3,None,None,NaN,NaN,NaN,NaN,"{""x"": 149.13494419931544, ""y"": -35.18550910935..."
3,4,None,None,NaN,NaN,NaN,NaN,"{""x"": 149.0674445255445, ""y"": -35.240087649525..."
4,5,None,None,NaN,NaN,NaN,NaN,"{""x"": 149.06936027755478, ""y"": -35.41792646263..."
5,6,None,None,NaN,NaN,NaN,NaN,"{""x"": 149.2338511028462, ""y"": -35.352636924054..."


In [46]:
sdf.to_csv("C:/Users/gjames/OneDrive - Esri Australia/Honours/Data/Testing/Services\\CQ_TownCentres.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/gjames/OneDrive - Esri Australia/Honours/Data/Testing/Services\\CQ_TownCentres.csv'

In [24]:
mylist = [1,2,3,4]

In [26]:
mydf = pd.DataFrame(mylist)

In [38]:
type(sdf)

TypeError: _repr_html_() missing 1 required positional argument: 'self'

In [15]:
sdf["SHAPE"]

0    {"x": 149.08546049210804, "y": -35.34473202895...
1    {"x": 149.1296045511092, "y": -35.276486929628...
2    {"x": 149.13494419931544, "y": -35.18550910935...
3    {"x": 149.0674445255445, "y": -35.240087649525...
4    {"x": 149.06936027755478, "y": -35.41792646263...
5    {"x": 149.2338511028462, "y": -35.352636924054...
Name: SHAPE, dtype: geometry

In [16]:
sdf["SHAPE"][0]["x"]

149.08546049210804

In [1]:
#define variables/paths
MB16_CQ_pop = "MB16_CQ_pop"
MB_test = "MB_test"
town_centres = "Services\\CQ_TownCentres"

current_path = "C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\TimeDistance_to_service\\TimeDistance_to_service.gdb"

In [10]:
#running closest facility
closest_facility = Closest_Facility_To_Centres(Incidents_Polygons = MB_test, Input_Facilities = town_centres)


In [11]:
TimeDistanceToFacility(Routes="Closest Facility\\Routes", 
                       Mesh_Block_Inputs=MB_test, 
                       Transfer_Fields=["Total_PublicTransitTime", "Total_Length"], 
                       Output_Path=current_path, 
                       Output_Name="MB16_Time_Distance")  # TimeDistance_To_Facility

In [12]:
#create an initial dataset to put the found times and distances in
Output_Name = "MB16_Time_Distance"
Base_layer = "MB16_CQ_pop"
MB16_Time_Distance = arcpy.conversion.FeatureClassToFeatureClass(in_features=Base_layer,
                                                                 out_path=current_path, 
                                                                 out_name=Output_Name,
                                                                 where_clause="", 
                                                                 config_keyword="")[0]

In [75]:
MB16_Time_Distance = arcpy.conversion.FeatureClassToFeatureClass(in_features=MB_test,
                                                                 out_path=current_path, 
                                                                 out_name=Output_Name,
                                                                 where_clause="", 
                                                                 config_keyword="")[0]

In [14]:
services_list = ["Services\\CQ_TownCentres", "Services\\CQ_GroupCentres"]

In [15]:

all_services_TimeDistance(services_list, MB_Layer = MB16_Time_Distance)

In [54]:
services_list[0].split("\\")[-1]

'CQ_TownCentres'

In [5]:
mystr = "mystr"

In [6]:
mystr[:18]

'mystr'

In [8]:
def TimeDistanceToFacility(Routes="Closest Facility\\Routes", Mesh_Block_Inputs="MB16_CQ_pop", Transfer_Fields=["Total_PublicTransitTime", "Total_Length"], Output_Path="C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\TestProject\\TestProject.gdb", Output_Name="MB16_Time_Distance"):  # TimeDistance_To_Facility

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Process: Feature Class To Feature Class (Feature Class To Feature Class) (conversion)
    #creates a new meshblock layer to put the data in
    MB16_Time_Distance = arcpy.conversion.FeatureClassToFeatureClass(in_features=Mesh_Block_Inputs, out_path=Output_Path, out_name=Output_Name, where_clause="", config_keyword="")[0]

    # Process: Join Field (Join Field) (management)
    #join the route data to the newly created meshblock data
    MB16_Accessability = arcpy.management.JoinField(in_data=MB16_Time_Distance, in_field="OBJECTID", join_table=Routes, join_field="IncidentID", fields=Transfer_Fields)[0]

In [30]:
#original working data production
def OD_data_production(layers, Mode, Network_Dataset):
    '''
    Finds the travel times between each of the given origins (MB data) and destinations (various service data) using the 
    Origin Destination Cost Matrix tool.
    
    Inputs: layers - a list of the names of point data to run the tool on (list of strings), note - these layers must be contained
            MainDatasets.gdb example: layers = ["CQ_TownCentres", "CQ_GroupCentres"]. Could implement an optional path specification
            
            Network_Dataset - the path to the network dataset to be used either of:
                "C:/Users/gjames/Documents/ArcGIS/Projects/Paths_NetworkDataset/Paths_NetworkDataset.gdb/Paths_ND/Paths_ND"
                "C:/Users/gjames/Documents/ArcGIS/Projects/GTFS_Integrated/GTFS_Integrated.gdb/GTFS_CQ/TransitNetwork_ND"
                
            Mode - The mode to be used in the analysis ("Walking", "Cycling" or "Public transit time")
            
    Outputs: A .csv file for each of the input layers, in the Honours/Data/OD Data Production folder
    '''
    
    arcpy.env.overwriteOutput = True #always important to include - allows tool to be run more than once without creating extra layers
    
    #initialise the network analysis layer
    Mode = "Walking"
    Network_Dataset = r"C:\Users\gjames\Documents\ArcGIS\Projects\Paths_NetworkDataset\Paths_NetworkDataset.gdb\Paths_ND\Paths_ND"
    
        
    
    arcpy.na.MakeODCostMatrixAnalysisLayer(Network_Dataset, "OD Cost Matrix", Mode, None, None, None, "LOCAL_TIME_AT_LOCATIONS", "STRAIGHT_LINES", None, "SKIP")
    
    #add the population data (constant for all runs)
    arcpy.na.AddLocations("OD Cost Matrix", "Origins", 
                          r"C:\Users\gjames\Documents\ArcGIS\Projects\MainDatasets\MainDatasets.gdb\MB16_CQ_pop_points", 
                          "Name # #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Walk_Time # #;Cutoff_Length # #;Cutoff_Cycle_Time # #",
                          "5000 Meters", None, "Intrg_CQ_roads SHAPE;Intrg_cyclepaths4 SHAPE;Intrg_footpaths SHAPE;Intrg_footpath_connectivity SHAPE;Intrg_resi_streets SHAPE;Paths_ND_Junctions NONE",
                          "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", None)
    
    #run the process for each of the layers
    for layer in layers:
        
        #add locations
        destinations_path = "C:/Users/gjames/Documents/ArcGIS/Projects/MainDatasets/MainDatasets.gdb/" + layer
        
        arcpy.na.AddLocations("OD Cost Matrix", "Destinations",
                              destinations_path,
                              "Name # #;CurbApproach # 0", "5000 Meters", None,
                              "Intrg_CQ_roads SHAPE;Intrg_cyclepaths4 SHAPE;Intrg_footpaths SHAPE;Intrg_footpath_connectivity SHAPE;Intrg_resi_streets SHAPE;Paths_ND_Junctions NONE",
                              "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", None)
        
        #solve
        arcpy.na.Solve("OD Cost Matrix", "SKIP", "TERMINATE", None, '')
        
        #export to csv
        output_name = "Lines_" + Mode + "__" + layer + ".csv"
        output_location = r"C:\Users\gjames\OneDrive - Esri Australia\Honours\Data\OD Data Production"
        
        arcpy.conversion.TableToTable(r"OD Cost Matrix\Lines", output_location, output_name, '',
                                      r'Name "Name" true true true 1024 Text 0 0,First,#,OD Cost Matrix\Lines,Name,0,1024;OriginID "OriginID" true true true 4 Long 0 0,First,#,OD Cost Matrix\Lines,OriginID,-1,-1;DestinationID "DestinationID" true true true 4 Long 0 0,First,#,OD Cost Matrix\Lines,DestinationID,-1,-1;DestinationRank "DestinationRank" true true true 4 Long 0 0,First,#,OD Cost Matrix\Lines,DestinationRank,-1,-1;Total_Walk_Time "Total_Walk_Time" true true true 8 Double 0 0,First,#,OD Cost Matrix\Lines,Total_Walk_Time,-1,-1;Total_Length "Total_Length" true true true 8 Double 0 0,First,#,OD Cost Matrix\Lines,Total_Length,-1,-1;Total_Cycle_Time "Total_Cycle_Time" true true true 8 Double 0 0,First,#,OD Cost Matrix\Lines,Total_Cycle_Time,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,OD Cost Matrix\Lines,Shape_Length,-1,-1', '')
    
    